In [1]:
from __future__ import division
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.contrib.layers import fully_connected

In [2]:
#prepare data

In [3]:
max_seq_length=30
trainDF = pd.read_csv('C:/Users/Administrator.NBJXUEJUN-LI/Desktop/project/RProj/my code/jdProj/save/traindf.csv',
                      header=0,encoding='gbk' )
testDF = pd.read_csv('C:/Users/Administrator.NBJXUEJUN-LI/Desktop/project/RProj/my code/jdProj/save/testdf.csv',
                      header=0,encoding='gbk' )
predDF = pd.read_csv('C:/Users/Administrator.NBJXUEJUN-LI/Desktop/project/RProj/my code/jdProj/save/preddf.csv',
                      header=0,encoding='gbk' )
 

In [4]:
prodmap= pd.read_csv('C:/Users/Administrator.NBJXUEJUN-LI/Desktop/project/RProj/my code/jdProj/save/prodEmbIndxMP.csv',
                      header=0,encoding='gbk' )
usermap= pd.read_csv('C:/Users/Administrator.NBJXUEJUN-LI/Desktop/project/RProj/my code/jdProj/save/userEmbIndxMP.csv',
                      header=0,encoding='gbk' )
brandmap= pd.read_csv('C:/Users/Administrator.NBJXUEJUN-LI/Desktop/project/RProj/my code/jdProj/save/brandEmbIndxMP.csv',
                      header=0,encoding='gbk' )
targmap= pd.read_csv('C:/Users/Administrator.NBJXUEJUN-LI/Desktop/project/RProj/my code/jdProj/save/targProdEmbIndxMP.csv',
                      header=0,encoding='gbk' )

prodNum = prodmap.index[-1]+1
userNum = usermap.index[-1]+1
brandNum = brandmap.index[-1]+1
targNum = targmap.index[-1]+1

prodmap=dict(zip(prodmap.mp ,prodmap.index))
usermap=dict(zip(usermap.mp ,usermap.index))
brandmap=dict(zip(brandmap.mp ,brandmap.index))
targmap=dict(zip(targmap.mp,targmap.index))


In [5]:
saticDefault = ['0']*23
def indexSplit(index,max_seq_length,mapdict):
    tmp = index.split("<-")[::-1]
    ret = [mapdict.get(int(tmp[i]),0) if i<len(tmp)else 0 for i in range(max_seq_length)]
    return ret
def staticFeatSplit(staticFeat,max_seq_length):
    tmp = staticFeat.split("<-")[::-1]
    ret = [tmp[i].split("#") if i<len(tmp)else saticDefault for i in range(max_seq_length)]
    return ret

In [6]:
trainProdSeq = np.array([indexSplit(prod,max_seq_length,prodmap) for prod in trainDF.skuidx],dtype=np.int32)
trainUserSeq = np.array([[usermap.get(user,0)]*max_seq_length for user in trainDF.user_id],dtype=np.int32)
trainBrandSeq = np.array([indexSplit(brand,max_seq_length,brandmap) for brand in trainDF.brandidx],dtype=np.int32)
trainStaticFeatSeq = np.array([staticFeatSplit(staticFeat,max_seq_length) for staticFeat in trainDF.static_feat],dtype=np.float32)
ySeq = np.array([targmap.get(int(label),0) for label in trainDF.label],dtype=np.int32)
trainSeqLength = np.array(trainDF.seq_length,dtype=np.int32)

testProdSeq = np.array([indexSplit(prod,max_seq_length,prodmap) for prod in testDF.skuidx],dtype=np.int32)
testUserSeq = np.array([[usermap.get(user,0)]*max_seq_length for user in testDF.user_id],dtype=np.int32)
testBrandSeq = np.array([indexSplit(brand,max_seq_length,brandmap) for brand in testDF.brandidx],dtype=np.int32)
testStaticFeatSeq = np.array([staticFeatSplit(staticFeat,max_seq_length) for staticFeat in testDF.static_feat],dtype=np.float32)
ySeqtest = np.array([targmap.get(int(label),0) for label in testDF.label],dtype=np.int32)
testSeqLength = np.array(testDF.seq_length,dtype=np.int32)

predProdSeq = np.array([indexSplit(prod,max_seq_length,prodmap) for prod in predDF.skuidx],dtype=np.int32)
predUserSeq = np.array([[usermap.get(user,0)]*max_seq_length for user in predDF.user_id],dtype=np.int32)
predBrandSeq = np.array([indexSplit(brand,max_seq_length,brandmap) for brand in predDF.brandidx],dtype=np.int32)
predStaticFeatSeq = np.array([staticFeatSplit(staticFeat,max_seq_length) for staticFeat in predDF.static_feat],dtype=np.float32)
predSeqLength = np.array(predDF.seq_length,dtype=np.int32) 

In [7]:
#set hyperparameters

In [8]:
alpha=0.001

In [9]:
#construct the graph

In [10]:
Prod_size  = prodNum
user_size  = userNum
brand_size = brandNum

prod_embedding_size  =20
user_embedding_size  =20
brand_embedding_size =20

max_step_num    = max_seq_length
static_feat_dim = 23
n_neurons       = 50

n_output        = targNum

In [11]:
prod_embeddings  = tf.Variable(tf.random_uniform([Prod_size,prod_embedding_size], -1.0, 1.0))
user_embeddings  = tf.Variable(tf.random_uniform([user_size,user_embedding_size], -1.0, 1.0))
brand_embeddings = tf.Variable(tf.random_uniform([brand_size,brand_embedding_size], -1.0, 1.0))

In [12]:
prod_idx  = tf.placeholder(tf.int32, shape=[None, max_step_num])
user_idx  = tf.placeholder(tf.int32, shape=[None, max_step_num])
brand_idx = tf.placeholder(tf.int32, shape=[None, max_step_num])

seq_length = tf.placeholder(tf.int32, [None])

static_feat = tf.placeholder(tf.float32, shape=[None, max_step_num,static_feat_dim])
prod_embed  = tf.nn.embedding_lookup(prod_embeddings , prod_idx)
user_embed  = tf.nn.embedding_lookup(user_embeddings , user_idx)
brand_embed = tf.nn.embedding_lookup(brand_embeddings, brand_idx)

y = tf.placeholder(tf.int32,[None])

In [13]:
final_feat=tf.concat([static_feat,prod_embed,user_embed,brand_embed], 2) 

In [14]:
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
 
basic_cell = tf.contrib.rnn.DropoutWrapper(basic_cell,input_keep_prob=0.5)

In [15]:
outputs, states = tf.nn.dynamic_rnn(basic_cell, final_feat, dtype=tf.float32,sequence_length=seq_length)
logits = fully_connected(states, n_output, activation_fn=None)

In [16]:
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,logits=logits)
loss = tf.reduce_mean(xentropy)

In [17]:
optimizer = tf.train.AdamOptimizer(learning_rate=alpha)
training_op = optimizer.minimize(loss)

In [18]:
correct = tf.nn.in_top_k(logits,y,1)
accuracy = tf.reduce_mean(tf.cast(correct,tf.float32))

In [19]:
init = tf.global_variables_initializer()

In [20]:
#feed data and train

In [21]:
saver = tf.train.Saver()

In [22]:
n_epochs=1000
batch_size=10
batch_num = len(ySeq)//batch_size
sessMode  = "init_train"
load_path = "old"
save_path = "old"

In [23]:
with tf.Session() as sess:
    if sessMode=="init_train":
        init.run()
        for epoch in range(n_epochs):
            for batch in range(batch_num):
                prod_batch       = trainProdSeq[batch*batch_size:(batch+1)*batch_size]
                user_batch       = trainUserSeq[batch:batch+batch_size]
                brand_batch      = trainBrandSeq[batch:batch+batch_size]
                static_batch     = trainStaticFeatSeq[batch:batch+batch_size]
                seq_length_batch = trainSeqLength[batch:batch+batch_size]
                y_batch          = ySeq[batch:batch+batch_size]
                sess.run(training_op, feed_dict={
                    prod_idx: prod_batch,user_idx:user_batch,brand_idx:brand_batch,static_feat:static_batch, 
                    seq_length: seq_length_batch,y:y_batch})
            if epoch%100==0:
                train_acc = accuracy.eval(feed_dict={
                        prod_idx: trainProdSeq,user_idx:trainUserSeq,brand_idx:trainBrandSeq,static_feat:trainStaticFeatSeq, 
                        seq_length: trainSeqLength,y:ySeq})
                test_acc  = accuracy.eval(feed_dict={
                        prod_idx: testProdSeq,user_idx:testUserSeq,brand_idx:testBrandSeq,static_feat:testStaticFeatSeq, 
                        seq_length: testSeqLength,y:ySeqtest})
                print(train_acc,test_acc)
        save_path = saver.save(sess,"/Users/snakepointid/Documents/project/RProj/my code/r相关函数/save/"+save_path+"/test.ckpt")
    elif sessMode=="continue_train":
        saver.restore(sess,"/Users/snakepointid/Documents/project/RProj/my code/r相关函数/save/"+load_path+"/test.ckpt")
        for epoch in range(n_epochs):
            nt,accuracy_eval = sess.run([training_op,accuracy], feed_dict={
                    prod_idx: prod_batch,user_idx:user_batch,brand_idx:brand_batch,static_feat:static_batch, 
                    seq_length: seq_length_batch,y:y_batch})
            print(accuracy_eval)
        save_path = saver.save(sess,"/Users/snakepointid/Documents/project/RProj/my code/r相关函数/save/"+save_path+"/test.ckpt")
    elif sessMode=="predict":
        saver.restore(sess,"/Users/snakepointid/Documents/project/RProj/my code/r相关函数/save/"+load_path+"/test.ckpt")
        logits_eval = logits.eval(feed_dict={
                    prod_idx: prod_batch,user_idx:user_batch,brand_idx:brand_batch,static_feat:static_batch, 
                    seq_length: seq_length_batch,y:y_batch})
        print(logits_eval.argmax(1))
    else:
        print("wrong session type")

0.0126263 0.0
0.128788 0.0
0.128788 0.010101
0.121212 0.010101
0.123737 0.020202
0.123737 0.020202
0.123737 0.020202
0.126263 0.020202
0.126263 0.030303
0.123737 0.030303


ValueError: Parent directory of /Users/snakepointid/Documents/project/RProj/my code/r相关函数/save/old/test.ckpt doesn't exist, can't save.